In [2]:
import os
import tensorflow as tf
from sklearn.model_selection import train_test_split
from tensorflow.keras import layers
from tensorflow import keras
import tensorflow as tf

from sklearn.model_selection import train_test_split
from ast import literal_eval

import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

#### Get the file name of data

In [3]:
data_dir = 'data'

folder, dirs, filenames = next(os.walk(data_dir))
filenames = [os.path.join(data_dir, name) for name in filenames] 
filenames[:5], len(filenames)

(['data\\001.txt',
  'data\\002.txt',
  'data\\003.txt',
  'data\\004.txt',
  'data\\005.txt'],
 209)

#### Splitting the data into train, test and validation sets and printing the results

In [4]:
# Split data by file into list format
# train_files, val_and_test_files = train_test_split(filenames, test_size = 0.2, random_state=42)
# val_files, test_files = train_tes
# t_split(val_and_test_files, test_size= 0.5, random_state=42)
# len(train_files), len(test_files), len(val_files)

#### Finding the vocabulary size: < word level embedding >

In [5]:
def get_vocabulary_size(filenames): 
    for filename in filenames:
        doc = []
        with open(filename, 'r') as f:
            for row in f:
                doc.append(row.lower())
            doc = " ".join(doc).split()
    return doc
n_vocab = len(set(get_vocabulary_size(train_files)))
# print(n_vocab)
# window_size = 10

NameError: name 'train_files' is not defined

#### create window dataset in word level

In [9]:
# import tensorflow as tf

# dataset = tf.data.Dataset.from_tensor_slices(tf.range(10))
# dataset = dataset.window(5, shift=1, drop_remainder=True)
# dataset = dataset.flat_map(lambda window: window.batch(5))
# dataset = dataset.map(lambda window: (window[:-1], window[-1:]))

def generate_window_dataset(filenames, window_size=10, shuffle=False, batch_size=32):
    documents = []
    for f in filenames:
        doc = tf.io.read_file(f)
        doc = tf.strings.lower(doc) # convert to lower case
        doc = tf.strings.regex_replace(doc,"\n"," ") # replacing new line with space
        doc = tf.strings.regex_replace(doc, "\r", "") # replcaing \r charter
        doc = tf.strings.split(doc, ' ') # split to word level 
        documents = documents + doc.numpy().tolist() # make dict word all file
    print(type(documents))
    doc_dataset = tf.data.Dataset.from_tensor_slices(documents) # load data into tensor from
    
    # make generate windows from list documents 
    # E.x. ['A', 'B', 'C', 'D', 'E', 'F', 'G']
#     doc_dataset = doc_dataset.window(size=window_size + 1, shift=1, drop_remainder=True)
    
#     # E.x. [['A', 'B', 'C', 'D', 'E', 'F', 'G']]
#     doc_dataset = doc_dataset.flat_map(lambda x: x.batch(window_size + 1))
    
#     # E.x. ['A', 'B', 'C', 'D', 'E', 'F'] => [G]    
#     #      ['B', 'C', 'D', 'E', 'F', 'G'] => [K]
#     doc_dataset = doc_dataset.map(lambda x: (x[:-1] , x[-1:]))
    
#     # shuffle the dataset with window size
#     if shuffle:
#         doc_dataset = doc_dataset.shuffle(len(list(doc_dataset)))
    
#     doc_dataset = doc_dataset.batch(batch_size)
#     doc_dataset = doc_dataset.prefetch(batch_size)
    
    return doc_dataset , documents


In [136]:
# train_ds, train_doc = generate_window_dataset(train_files)
# test_ds, test_doc = generate_window_dataset(test_files)
# val_ds, val_doc = generate_window_dataset(val_files)

In [11]:
data_set, doc_set = generate_window_dataset(filenames,shuffle=True)

<class 'list'>


In [138]:
# preview data set
# [f"{x[0]} ----- {x[1]}" for x in data_set.take(2)]

In [139]:
def split_data_set(data_sets):
    X = []
    y = []
    for val in data_sets:
        X.append(val[0])
        y.append(val[1])
    return X, y

In [7]:
data_set

<PrefetchDataset element_spec=(TensorSpec(shape=(None, None), dtype=tf.string, name=None), TensorSpec(shape=(None, None), dtype=tf.string, name=None))>

In [8]:
[(train_features, label_batch)] = data_set.take(1)
train_features[0], label_batch[0]

(<tf.Tensor: shape=(10,), dtype=string, numpy=
 array([b'was', b'running', b'away', b'at', b'full', b'speed,', b'and',
        b'high', b'up', b'on'], dtype=object)>,
 <tf.Tensor: shape=(1,), dtype=string, numpy=array([b'the'], dtype=object)>)

### Create an input pipeline using tf.data

### Implementing the language model:

#### First defining the tokenization layer and integrating it into the model

In [125]:
# X_train, y_train = split_data_set(train_ds)
# X_val, y_val = split_data_set(val_ds)

In [141]:
data_set, label = split_data_set(data_set)
data_set[0], label[0]

(<tf.Tensor: shape=(32, 10), dtype=string, numpy=
 array([[b'shall', b'not', b'return', b'for', b'three', b'days.', b'',
         b'if', b'during', b'that'],
        [b'blown', b'out', b'by', b'the', b'wind.', b'she', b'ran',
         b'back', b'home,', b'trembling'],
        [b'the', b'hearth.', b'', b'then', b'they', b'went', b'to',
         b'the', b'needle', b'which'],
        [b'into', b'the', b'kitchen', b'and', b'wanted', b'to', b'light',
         b'a', b'pipe,', b'but'],
        [b'hans', b'gave', b'her', b'a', b'hare', b'into', b'her',
         b'apron,', b'but', b'when'],
        [b'would', b'lie', b'on', b'the', b'ground,', b'secondly,', b'of',
         b'a', b'cloak', b'which'],
        [b'fox.', b'', b'you', b'know', b'how', b'to', b'find', b'all',
         b'kinds', b'of'],
        [b'night', b'outside', b'the', b"king's", b"son's", b'door,',
         b'the', b'bride', b'might', b'have'],
        [b'his', b'fate,', b'and', b'crawl', b'along', b'patiently',
         b'behi

In [142]:
len(label)

9237

In [145]:
# Multi-label binarization
# terms = tf.ragged.constant(y_t)
# lookup = tf.keras.layers.StringLookup(output_mode="multi_hot")
# lookup.adapt(terms)
# vocab = lookup.get_vocabulary()
# vocab

terms = label[0].numpy().copy()
lookup = tf.keras.layers.StringLookup(output_mode="multi_hot")
lookup.adapt(terms)
vocab = lookup.get_vocabulary()


def invert_multi_hot(encoded_labels):
    """Reverse a single multi-hot encoded label to a tuple of vocab terms."""
    hot_indices = np.argwhere(encoded_labels == 1.0)[..., 0]
    return np.take(vocab, hot_indices)


print("Vocabulary:\n")
print(vocab)

Vocabulary:

['[UNK]', 'and', 'what', '', 'you', 'woman', 'when', 'was', 'walked', 'time', 'then', 'the', 'she', 'put', 'made', 'like', 'laid', 'it', 'in', 'i', 'how', 'him', 'hiding-places,', 'her', 'he,', 'got', 'another', 'a']


In [146]:
import tensorflow.keras.utils as utils
c = utils.to_categorical([1,2,1,2,1,2,1,3,1,3], num_classes=1000)
c.shape

(10, 1000)

In [148]:
lookup([terms[0]])

<tf.Tensor: shape=(1, 28), dtype=float32, numpy=
array([[0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]], dtype=float32)>

In [88]:
y_t = [z.numpy()[0] for z in terms]

In [89]:
y_t[:5]

[b'', b'and', b'and', b'of', b'nothing']

In [91]:
# lookup()

In [50]:
lookup = tf.keras.layers.StringLookup(output_mode="multi_hot")
lookup.adapt([[terms]])
vocab = lookup.get_vocabulary()


In [96]:
len(terms)

9237

In [94]:
label_binarizedz

<tf.Tensor: shape=(1, 2221), dtype=float32, numpy=array([[0., 1., 1., ..., 1., 1., 1.]], dtype=float32)>

In [99]:
import tensorflow.keras.layers as layers
import tensorflow.keras.models as models
import tensorflow.keras.backend as K

text_vectorizer = layers.TextVectorization(max_tokens=2221, output_mode="int", output_sequence_length=10, standardize=None, split=None, input_shape=(10, ))


In [100]:

# train model on data
vectorize_layer = text_vectorizer.adapt(X_train)

NameError: name 'X_train' is not defined

In [34]:
text_vectorizer([b'the', b'cock', b'once', b'said', b'to', b'the', b'hen,', b'it',
        b'is', b'now'])

<tf.Tensor: shape=(10,), dtype=int64, numpy=array([ 2, 15, 50, 26,  5,  2, 49,  7, 48, 21], dtype=int64)>

In [102]:
# Creating an Embedding using an Embedding Layer
tf.random.set_seed(42)

embedding = layers.Embedding(input_dim=2221,
                             output_dim=128,
                             embeddings_initializer="uniform",
                             input_length=10, 
                             name="embedding_1")


In [36]:
sample_embed = embedding(text_vectorizer([b'the', b'cock', b'once', b'said', b'to', b'the', b'hen,', b'it',
        b'is', b'now']))
sample_embed[0]

<tf.Tensor: shape=(128,), dtype=float32, numpy=
array([-0.03123627,  0.00086542,  0.04846634,  0.02067694,  0.04488743,
        0.02432524,  0.00755162,  0.01100441,  0.0486321 ,  0.00958809,
       -0.04868646,  0.03798235, -0.01554748, -0.04864894, -0.03886639,
       -0.02819755, -0.01342027, -0.02886009, -0.01321923,  0.01613373,
       -0.03610392, -0.0390061 , -0.00761154, -0.01587026,  0.01349435,
        0.04889158,  0.04427209, -0.02655149, -0.02685558,  0.01160132,
        0.02260213, -0.03419708, -0.02412816, -0.0372295 , -0.03792644,
       -0.00437276, -0.00885792,  0.02040186,  0.02619821,  0.02895229,
       -0.02212901, -0.00365621, -0.00859733,  0.03556058, -0.04759806,
       -0.01372441,  0.01025496,  0.01947612, -0.04064914,  0.00463156,
        0.00067716, -0.01626496, -0.04619929,  0.02992364,  0.00688486,
       -0.04084289,  0.01401979,  0.03595698, -0.03698256,  0.03336043,
        0.03453244, -0.03492796, -0.0383046 , -0.01607203,  0.03766021,
        0.006783

In [ ]:
# Create LSTM model
inputs = layers.Input(shape=(1,), dtype="string")
# x = text_vectorizer(inputs)
x = embedding(inputs)
x = layers.LSTM(64)(x)
outputs = layers.Dense(1, activation="softmax")(x)
model_2 = tf.keras.Model(inputs, outputs, name="model_2_LSTM")

In [88]:
model_2.compile(loss="categorical_crossentropy",
               optimizer=tf.keras.optimizers.Adam(),
               metrics=["accuracy"])

In [89]:
model_2.summary()

Model: "model_2_LSTM"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_3 (InputLayer)        [(None, 1)]               0         
                                                                 
 embedding_1 (Embedding)     multiple                  65920     
                                                                 
 lstm_8 (LSTM)               (None, 64)                49408     
                                                                 
 dense_8 (Dense)             (None, 1)                 65        
                                                                 
Total params: 115,393
Trainable params: 115,393
Non-trainable params: 0
_________________________________________________________________


In [90]:
model_2_history = model_2.fit(X_tr, label, epochs=5, validation_data=(X_val, y_val))

Epoch 1/5


ValueError: in user code:

    File "c:\users\ntviet5\desktop\tensorflow-certificate\env\lib\site-packages\keras\engine\training.py", line 1249, in train_function  *
        return step_function(self, iterator)
    File "c:\users\ntviet5\desktop\tensorflow-certificate\env\lib\site-packages\keras\engine\training.py", line 1233, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "c:\users\ntviet5\desktop\tensorflow-certificate\env\lib\site-packages\keras\engine\training.py", line 1222, in run_step  **
        outputs = model.train_step(data)
    File "c:\users\ntviet5\desktop\tensorflow-certificate\env\lib\site-packages\keras\engine\training.py", line 1024, in train_step
        loss = self.compute_loss(x, y, y_pred, sample_weight)
    File "c:\users\ntviet5\desktop\tensorflow-certificate\env\lib\site-packages\keras\engine\training.py", line 1083, in compute_loss
        y, y_pred, sample_weight, regularization_losses=self.losses
    Fil

In [42]:
len(X_train) , len(y_train), len(X_val), len(y_val)

(234423, 234423, 29795, 29795)

In [45]:
train_dataset = text_vectorizer.adapt()

<tf.Tensor: shape=(10,), dtype=string, numpy=
array([b'the', b'cock', b'once', b'said', b'to', b'the', b'hen,', b'it',
       b'is', b'now'], dtype=object)>

In [ ]:
text_vectorizer.get_vocabulary()

In [96]:
X_val[16]

<tf.Tensor: shape=(10,), dtype=string, numpy=
array([b'then', b'the', b'little', b'louse', b'fell', b'in', b'and',
       b'burnt', b'herself.', b''], dtype=object)>

In [100]:
X_train[0], y_train[0]

(<tf.Tensor: shape=(10,), dtype=string, numpy=
 array([b'the', b'cock', b'once', b'said', b'to', b'the', b'hen,', b'it',
        b'is', b'now'], dtype=object)>,
 <tf.Tensor: shape=(1,), dtype=string, numpy=array([b'the'], dtype=object)>)

In [101]:
X_train

[<tf.Tensor: shape=(10,), dtype=string, numpy=
 array([b'the', b'cock', b'once', b'said', b'to', b'the', b'hen,', b'it',
        b'is', b'now'], dtype=object)>,
 <tf.Tensor: shape=(10,), dtype=string, numpy=
 array([b'cock', b'once', b'said', b'to', b'the', b'hen,', b'it', b'is',
        b'now', b'the'], dtype=object)>,
 <tf.Tensor: shape=(10,), dtype=string, numpy=
 array([b'once', b'said', b'to', b'the', b'hen,', b'it', b'is', b'now',
        b'the', b'time'], dtype=object)>,
 <tf.Tensor: shape=(10,), dtype=string, numpy=
 array([b'said', b'to', b'the', b'hen,', b'it', b'is', b'now', b'the',
        b'time', b'when'], dtype=object)>,
 <tf.Tensor: shape=(10,), dtype=string, numpy=
 array([b'to', b'the', b'hen,', b'it', b'is', b'now', b'the', b'time',
        b'when', b'the'], dtype=object)>,
 <tf.Tensor: shape=(10,), dtype=string, numpy=
 array([b'the', b'hen,', b'it', b'is', b'now', b'the', b'time', b'when',
        b'the', b'nuts'], dtype=object)>,
 <tf.Tensor: shape=(10,), dtype=st

In [71]:
label = text_vectorizer(y_train)

In [79]:
label.shape

TensorShape([234423, 10])

In [80]:
X_tr = text_vectorizer(X_train)

In [84]:
X_tr.shape

TensorShape([234423, 10])

In [46]:
import numpy as np
x = np.zeros((32,10), dtype=np.int32)
y = np.zeros((32), dtype=np.int32)

In [78]:
y_train

[<tf.Tensor: shape=(1,), dtype=string, numpy=array([b'the'], dtype=object)>,
 <tf.Tensor: shape=(1,), dtype=string, numpy=array([b'time'], dtype=object)>,
 <tf.Tensor: shape=(1,), dtype=string, numpy=array([b'when'], dtype=object)>,
 <tf.Tensor: shape=(1,), dtype=string, numpy=array([b'the'], dtype=object)>,
 <tf.Tensor: shape=(1,), dtype=string, numpy=array([b'nuts'], dtype=object)>,
 <tf.Tensor: shape=(1,), dtype=string, numpy=array([b'are'], dtype=object)>,
 <tf.Tensor: shape=(1,), dtype=string, numpy=array([b'ripe,'], dtype=object)>,
 <tf.Tensor: shape=(1,), dtype=string, numpy=array([b'so'], dtype=object)>,
 <tf.Tensor: shape=(1,), dtype=string, numpy=array([b'let'], dtype=object)>,
 <tf.Tensor: shape=(1,), dtype=string, numpy=array([b'us'], dtype=object)>,
 <tf.Tensor: shape=(1,), dtype=string, numpy=array([b'go'], dtype=object)>,
 <tf.Tensor: shape=(1,), dtype=string, numpy=array([b'to'], dtype=object)>,
 <tf.Tensor: shape=(1,), dtype=string, numpy=array([b'the'], dtype=object)>

In [47]:
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout, Bidirectional
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam
from tensorflow.keras import regularizers
import tensorflow.keras.utils as ku 
import numpy as np 
 
 
tokenizer = Tokenizer()
data = open('11-0.txt', encoding="utf-8").read()
 
corpus = data.lower().split("\n")
tokenizer.fit_on_texts(corpus)
total_words = len(tokenizer.word_index) + 1
 
# create input sequences using list of tokens
input_sequences = []
for line in corpus:
    token_list = tokenizer.texts_to_sequences([line])[0]
    for i in range(1, len(token_list)):
        n_gram_sequence = token_list[:i+1]
        input_sequences.append(n_gram_sequence)
 
 
# pad sequences 
max_sequence_len = max([len(x) for x in input_sequences])
input_sequences = np.array(pad_sequences(input_sequences, maxlen=max_sequence_len, padding='pre'))
 
# create predictors and label
predictors, label = input_sequences[:,:-1],input_sequences[:,-1]
 
label = ku.to_categorical(label, num_classes=total_words)
 
 
model = Sequential()
model.add(Embedding(total_words, 100, input_length=max_sequence_len-1))
model.add(Bidirectional(LSTM(150, return_sequences = True)))
model.add(Dropout(0.2))
model.add(LSTM(100))
model.add(Dense(total_words/2, activation='relu', kernel_regularizer=regularizers.l2(0.01)))
model.add(Dense(total_words, activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
print(model.summary())

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_2 (Embedding)     (None, 17, 100)           353400    
                                                                 
 bidirectional_2 (Bidirectio  (None, 17, 300)          301200    
 nal)                                                            
                                                                 
 dropout_2 (Dropout)         (None, 17, 300)           0         
                                                                 
 lstm_6 (LSTM)               (None, 100)               160400    
                                                                 
 dense_5 (Dense)             (None, 1767)              178467    
                                                                 
 dense_6 (Dense)             (None, 3534)              6248112   
                                                      

In [48]:
predictors

array([[   0,    0,    0, ...,    0,    0, 1826],
       [   0,    0,    0, ...,    0, 1826,   46],
       [   0,    0,    0, ..., 1826,   46,   41],
       ...,
       [   0,    0,    0, ..., 3533,    4,  279],
       [   0,    0,    0, ...,    4,  279,   38],
       [   0,    0,    0, ...,  279,   38,  506]])

In [49]:
label[0]

array([0., 0., 0., ..., 0., 0., 0.], dtype=float32)

In [51]:
history = model.fit(predictors, label, epochs=1, verbose=1)

881/881 [==============================] - 82s 93ms/step - loss: 6.0543 - accuracy: 0.0582


In [ ]:
seed_text = "alice was not a bit hurt"
next_words = 100
   
for _ in range(next_words):
    token_list = tokenizer.texts_to_sequences([seed_text])[0]
    token_list = pad_sequences([token_list], maxlen=max_sequence_len-1, padding='pre')
    predicted = model.predict_classes(token_list, verbose=0)
    output_word = ""
    for word, index in tokenizer.word_index.items():
        if index == predicted:
            output_word = word
            break
    seed_text += " " + output_word
print(seed_text)

In [52]:
predictors.shape , label.shape

((28186, 17), (28186, 3534))

In [ ]:
[print(x) for x in label[0]]

In [ ]:
label[0][46]

In [ ]:
predictors[3000]

In [ ]:
label[3000]

In [54]:
len(predictors[0])

17

In [56]:
predictors[1]

array([   0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0, 1826,   46])

In [57]:
input_sequences

array([[   0,    0,    0, ...,    0, 1826,   46],
       [   0,    0,    0, ..., 1826,   46,   41],
       [   0,    0,    0, ...,   46,   41,  314],
       ...,
       [   0,    0,    0, ...,    4,  279,   38],
       [   0,    0,    0, ...,  279,   38,  506],
       [   0,    0,    0, ...,   38,  506,  624]])

In [ ]:

from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout, Bidirectional
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam
from tensorflow.keras import regularizers
import tensorflow.keras.utils as ku 
import numpy as np 
 
 
tokenizer = Tokenizer()
data = open('11-0.txt', encoding="utf-8").read()
corpus = data.lower().split("\n")
tokenizer.fit_on_texts(corpus)

total_words = len(tokenizer.word_index) + 1


# create input sequences using list of tokens
input_sequences = []
for line in corpus[:3]:
    print(tokenizer)
    print(total_words)
    print(line)
    token_list = tokenizer.texts_to_sequences([line])[0]
    print(token_list)
    for i in range(1, len(token_list)):
        n_gram_sequence = token_list[:i+1]
        input_sequences.append(n_gram_sequence)
        
    print(input_sequences)
    
 
# pad sequences 
max_sequence_len = max([len(x) for x in input_sequences])
input_sequences = np.array(pad_sequences(input_sequences, maxlen=max_sequence_len, padding='pre'))
 
# create predictors and label
predictors, label = input_sequences[:,:-1],input_sequences[:,-1]
print(predictors)
 
label = ku.to_categorical(label, num_classes=total_words)
print(label)


In [58]:
 
tokenizer = Tokenizer()
data = open('11-0.txt', encoding="utf-8").read()
 
corpus = data.lower().split("\n")
tokenizer.fit_on_texts(corpus)
total_words = len(tokenizer.word_index) + 1
 
# create input sequences using list of tokens
input_sequences = []
for line in corpus:
    token_list = tokenizer.texts_to_sequences([line])[0]
    for i in range(1, len(token_list)):
        n_gram_sequence = token_list[:i+1]
        input_sequences.append(n_gram_sequence)
 

In [1]:
corpus

NameError: name 'corpus' is not defined

In [60]:
tokenizer.fit_on_texts(corpus)

In [13]:
import random

def getT(): 
    return random.randint(0, 7)
    
def getTX():
    return [getT(), getT(), getT()]

def predict():
    cus = getTX()
    tes = getTX()
    if sum(cus)> 10:
        if sum(tes)> 10:
            print('true')
        else:
            print('false')
    else:
        if sum(tes)< 11:
            print('true')
        else:
            print('false')

In [15]:
for x in range(500):
    predict()

false
false
false
true
false
true
true
false
true
true
false
false
true
true
true
true
false
true
false
false
false
true
true
false
false
true
false
true
false
true
true
false
false
true
false
false
false
false
true
false
true
false
false
false
false
false
true
false
true
false
true
false
false
true
true
false
true
true
false
false
true
true
true
true
true
false
true
true
true
false
false
false
false
false
false
false
false
true
true
true
true
true
true
false
false
true
true
true
true
false
true
true
true
false
false
false
true
false
false
true
false
true
false
false
false
true
false
true
true
true
true
false
true
true
true
false
true
true
false
false
true
true
true
true
false
false
true
false
false
true
false
true
false
true
false
false
false
true
true
false
false
false
true
true
true
false
true
false
true
false
false
false
false
false
true
false
false
true
true
true
false
false
true
true
false
false
false
false
false
true
false
false
false
false
false
true
false
true
false
true
true


In [8]:
[getT(), getT(), getT()]

[None, None, None]

In [2]:
10+10+30+60+130+300+600

1140